взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91' -O churn_data.csv

In [ ]:
df = pd.read_csv("churn_data.csv")
df.head(3)

In [ ]:
df['Exited'].value_counts()

In [ ]:
#СЂР°Р·РґРµР»РёРј РґР°РЅРЅС‹Рµ РЅР° train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Exited']), df['Exited'], random_state=0)

In [ ]:
df.head(3)

сделать обзорный анализ выбранного датасета

In [ ]:
cat_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
cont_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

X_train = X_train[cat_columns + cont_columns]
X_test = X_test[cat_columns + cont_columns]

## 3.обучить любой классификатор (какой вам нравится)

In [ ]:
!pip install catboost
from catboost import CatBoostClassifier

In [ ]:
model = CatBoostClassifier(random_state=42, n_estimators=50, cat_features=cat_columns)

In [ ]:
# РѕР±СѓС‡РёРј РЅР°С€ РїР°Р№РїР»Р°Р№РЅ
model.fit(X_train, y_train)

In [ ]:
# РЅР°С€Рё РїСЂРѕРіРЅРѕР·С‹ РґР»СЏ С‚РµСЃС‚РѕРІРѕР№ РІС‹Р±РѕСЂРєРё
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

## 4.  разобраться с SHAP и построить важности признаков для:
- всего тестового набора данных (summary_plot - дать интерпретацию)
- для топ 10%

In [ ]:
!pip install shap

In [ ]:
import shap

explainer = shap.TreeExplainer(model)

In [ ]:
preds_train = model.predict_proba(X_train)[:, 1]
preds_train

In [ ]:
%%time
shap_values_all = explainer.shap_values(X_train)

In [ ]:
features = cat_columns + cont_columns

In [ ]:
shap.summary_plot(shap_values_all, X_train, feature_names=features)

In [ ]:
# visualize the first prediction's explanation
shap.initjs()
idx = 1
print('pred is', preds_train[idx])
shap.force_plot(explainer.expected_value, shap_values_all[idx,:], features=X_train.iloc[idx,:], feature_names=features)

In [ ]:
# visualize the first prediction's explanation
shap.initjs()
idx = 2
print('pred is', preds_train[idx])
shap.force_plot(explainer.expected_value, shap_values_all[idx,:], features=X_train.iloc[idx,:], feature_names=features)

In [ ]:
# visualize the first prediction's explanation
shap.initjs()
idx = 7499
print('pred is', preds_train[idx])
shap.force_plot(explainer.expected_value, shap_values_all[idx,:], features=X_train.iloc[idx,:], feature_names=features)

In [ ]:
# visualize the first prediction's explanation
shap.initjs()
idx = 40
print('pred is', preds_train[idx])
shap.force_plot(explainer.expected_value, shap_values_all[idx,:], features=X_train.iloc[idx,:], feature_names=features)

In [ ]:
# visualize the first prediction's explanation
shap.initjs()
idx = 5644
print('pred is', preds_train[idx])
shap.force_plot(explainer.expected_value, shap_values_all[idx,:], features=X_train.iloc[idx,:], feature_names=features)